# PyKOALA Raw Stacked Spectra (RSS) 

## Table of contents:

1. [Importing class](#importing-class)
2. [Koala RSS](#koala-rss)
    - [`RSS` Attributes](#rss-attributes)
        - [General information](#general-information)
        - [History record](#history-record)
    - [Data methods](#data-methods)
        - [`is_corrected`](#is_corrected)
        - [`get_centre_of_mass`](#get_centre_of_mass)
        - [`update_coordinates`](#update_coordinates)
        - [`get_integrated_fibres`](#get_integrated_fibres)
        - [`to_fits`](#to_fits)
        - [`from_fits`](#from_fits)
    - [Plotting methods](#plotting-methods)
        - [`plot_rss_image`](#plot_rss_image)
        - [`plot_mask`](#plot_mask)
        - [`plot_fibres`](#plot_fibres)


## Importing class

**Note: Make sure to run the following cells in order to ensure correct execution.**

In [ ]:
from pykoala.data_container import RSS
import os
from pykoala.instruments.koala_ifu import koala_rss
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

## Koala RSS

We can extract the RSS data using the function `koala_rss` in `koala_ifu`

In [ ]:
data_path = '../data/koala/'
single_fits_sample = '385R/27feb20028red.fits'
file_sample = os.path.join(data_path, single_fits_sample)
rss_sample = koala_rss(file_sample)

### RSS attributes

#### General information 

Information about the RRS can be obtained from the `info` attribute:

In [ ]:
rss_sample.info

The information is given as a dictionary:

In [ ]:
rss_sample.info.keys()

Intensity values can be shown with: 

In [ ]:
rss_sample.intensity

Intensity is saved as a `astropy` quantity, so we can extract its unit as an attribute:

In [ ]:
units_intensity = rss_sample.intensity.unit
print(f'Unit of the intensity data is: {units_intensity}')

Similarly, with the variance of the RSS:

In [ ]:
units_variance = rss_sample.variance.unit

print(f'Variance: {rss_sample.variance}')
print('\n================================\n')
print(f'Unit of the intensity data is: {units_variance}')

#### History record

RSS metada is stored as a `DataContainerHistory` object

In [ ]:
rss_sample.history.show()

#### RSS fibres

Information about the fibres can be obtained from here:

In [ ]:
rss_sample.fibre_diameter

With `science_fibres` we can find the indexes of the fibres with non-bad pixels:

In [ ]:
number_of_non_bad_fibres = len(rss_sample.science_fibres)
print(f'Indexes of non-bad fibers: {rss_sample.science_fibres}')
print('\n================================\n')
print(f'Number of non-bad fibers: {number_of_non_bad_fibres}')

### Data methods

#### `is_corrected`

We can check if a correction has been done in the RSS data with the `is_corrected` method: 

In [ ]:
rss_sample.is_corrected(correction='read')

#### `get_centre_of_mass`

We can show the center of mass (COM) based on the RSS fibre positions:

In [ ]:
rss_sample.get_centre_of_mass()

The first and second lists contain ra and dec coordinates, respectively.

#### `update_coordinates`

We can modify the fibres' coordinates with this method. First, let's check the original coordinates:

In [ ]:
original_ra, original_dec = rss_sample.info["fib_ra"], rss_sample.info["fib_dec"]
print(f'Original RA fibres coordinates: {original_ra[:10]}')
print('\n================================\n')
print(f'Original DEC fibres coordinates: {original_dec[:10]}')


Now we use `update_coordinates`:

In [ ]:
rss_sample.update_coordinates(new_coords=(2*original_ra,2*original_dec))

In [ ]:
print(f'Updated RA fibres coordinates: {rss_sample.info["fib_ra"][:10]}')
print('\n================================\n')
print(f'Updated DEC fibres coordinates: {rss_sample.info["fib_dec"][:10]}')

#### `get_integrated_fibres`

We compute the integrated intensity of the RSS fibres with this method:

In [ ]:
rss_sample.get_integrated_fibres()

We can also constraint the calculation in a specific wavelength.

In [ ]:
rss_sample.wavelength

Now we select the wavelenght range. **Make sure to use astropy quantities and not simple float values.** 

In this case we will use the same elements of the RSS wavelengh data to define the range.

In [ ]:
rss_sample.get_integrated_fibres(wavelength_range=[6000,7000])

#### `to_fits`

We can write the RSS into a FITS file with this method.  

In [ ]:
path_output = './output/'
fits_file_path = path_output + 'rss_sample.fits'
rss_sample.to_fits(filename=fits_file_path,overwrite=True)

Notice that the format of the FITS file is different from the original.

#### `from_fits` 

This FITS format can be read with this method:

In [ ]:
new_rss_sample = RSS.from_fits(filename=fits_file_path)

Note that we call this method from the RSS class, as oposed as from the object like the previous methods shown here.

The new object has the same properties as `rss_sample`:

In [ ]:
new_rss_sample.info

In [ ]:
comparing_intensity_values = np.array_equal(new_rss_sample.intensity,rss_sample.intensity,equal_nan=True)
comparing_variance_values = np.array_equal(new_rss_sample.variance,rss_sample.variance,equal_nan=True)
print(f"Comparing intensity values: {comparing_intensity_values}")
print(f"Comparing variance values: {comparing_variance_values}")

### Plotting methods

We can directly do RSS related plots with the following methods  

#### `plot_rss_image`

Plot of basic RSS properties:

In [ ]:
rss_sample.plot_rss_image()
plt.show()

Custom version:

In [ ]:

plot_file_path = path_output + '/custom_plot_rss_image.png'
rss_sample.plot_rss_image(data=rss_sample.variance.value, data_label='Variance',fibre_range=(100,300),
                            wavelength_range=(6000,7000),output_filename=plot_file_path)
plt.show()

#### `plot_mask`

This method creates a plot of the bitmask data:

In [ ]:
rss_sample.plot_mask()
plt.plot()

#### `plot_fibres`

This methods plots a fibre map image, showing the spatial distribution of data across fibres.

In [ ]:
rss_sample.plot_fibres()
plt.show()